## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier

from batman import batman_curve, get_dataset
from utils import (
    bootstrap_sampler,
    visualize_data,
    compute_main_predictions,
    bias_variance_estimation,
    plot_contour
)

%load_ext autoreload
%autoreload 2

## Create random points

In [ ]:
sample_size = 10000
X1 = 16*np.random.random(size=(sample_size, 1)) - 8
X2 =  8*np.random.random(size=(sample_size, 1)) - 4
X = np.concatenate((X1, X2), axis=1)
Y = np.zeros((sample_size, 1))

for i in range(len(X)):
    curr_cat = batman_curve(X[i, 0], X[i, 1])
    Y[i, 0] = curr_cat

color = list(map(lambda x: 'y' if x == 1 else 'k', Y))

## Show scatter plot

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(X[:, 0], X[:, 1], color=color)

## Cleaner figure

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(X[:, 0], X[:, 1], color=color)
plt.axis('off')
plt.savefig('resources/batman-logo.png', dpi=300, bbox_inches='tight')

## Different data regimes (low, medium, high)

In [ ]:
regime_data = dict()
regime_sizes = [300, 1000, 3000]
regime_names = ['low', 'medium', 'high']

for name, size in zip(regime_names, regime_sizes):
    regime_data[name] = get_dataset(sample_size=size)

    X, Y, Y_star = regime_data[name]
    color_y      = list(map(lambda x: 'y' if x == 1 else 'k', Y))
    color_y_star = list(map(lambda x: 'y' if x == 1 else 'k', Y_star))

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

    axes[0].set_title('Noisy data')
    axes[0].scatter(X[:, 0], X[:, 1], color=color_y)

    axes[1].set_title('Noiseless data')
    axes[1].scatter(X[:, 0], X[:, 1], color=color_y_star)

## Simple classifier

In [ ]:
clf = KNeighborsClassifier(n_neighbors=1)
X, Y, Y_star = regime_data['medium']

clf.fit(X, Y[:, 0])

## Create grid data

In [ ]:
step = 0.1
grid_x1 = (-8, 8)
grid_x2 = (-4, 4)

# linearly spaced coordinates
xx1 = np.arange(grid_x1[0], grid_x1[1] + step, step)
xx2 = np.arange(grid_x2[0], grid_x2[1] + step, step)

XX = np.array(np.meshgrid(xx1, xx2)).T.reshape(-1, 2)

## Predictions on a regular grid

In [ ]:
YY = clf.predict(XX)

## Simple visualization of the results

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 5))

# Classifier results
visualize_data(XX, YY, axes[0], title='Classifier')

# Data
visualize_data(X, Y,      axes[1], title='Noisy (training) data')
visualize_data(X, Y_star, axes[2], title='Noiseless data')

## Compare classifier results according to different data regimes

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(5*len(regime_names), 5))
for regime_name, ax in zip(regime_names, axes):
    # get data
    X, Y, Y_star = regime_data[regime_name]

    # create and fit the classifier
    clf = KNeighborsClassifier(n_neighbors=1)
    clf.fit(X, Y[:, 0])

    # prediction
    prediction = clf.predict(XX)

    # visualization
    visualize_data(XX, prediction, ax, title=regime_name)

## Bias and variance estimation

### Inputs

In [ ]:
# Inputs
selected_regime = 'high'
data = regime_data[selected_regime]
num_bootstrap_samples = 100 # TODO: increase it later to at least 1000

grid = XX

### Main Function

In [ ]:
noises, biases, variances, losses, grid_predictions = \
    bias_variance_estimation(clf, data, num_bootstrap_samples, grid)

### Plot the results

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=1, figsize=(16, 8))
contour_grid = np.array(np.meshgrid(xx1, xx2))
plot_contour(contour_grid, grid_predictions, axes)